In [ ]:
# imports
%reload_ext autoreload
%autoreload 2
%matplotlib inline 

import sys
import pandas as pd
import numpy as np
from utils.basic_utils import config, read_dates, load_csvs, excl
from utils.pricing import get_left_right, load_px_close, get_symbol_pricing
from utils.pricing import apply_std_boundaries, freq_dist, rank_roll_perf
from utils.pricing import get_ind_index, to_index_form, eq_wgt_attribution
from utils.pricing import shorten_name
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.float_format = '{:,.2f}'.format
cmap = 'tab20'
fs, lw, alp, loc, margin = (12,5), 5, 0.5, 'center left', (1, 0.5)
sl = 3 # to slice large lists in top / bottom chunks
# pd.describe_option()

### General functions

In [ ]:
# # utility functions, should be in pricing.py already
# freq_dist = lambda df, col, tail: df[col].tail(tail).value_counts(bins=12, normalize=True).sort_index()
# shorten_name = lambda x: "".join([str.upper(z[:3]) for z in x])
# roll_vol = lambda df, rw: (df.rolling(rw).std() * pow(252, 1/2))

# # helper methods
# def build_px_struct(data_dict, freq):
#     dt = date if freq == '1d' else datetime
#     dates = [dt.fromtimestamp(x) for x in data_dict['timestamp']]
#     hist_pricing = data_dict['indicators']['quote'][0]
#     H = hist_pricing['high']
#     L = hist_pricing['low']
#     O = hist_pricing['open']
#     C = hist_pricing['close']
#     V = hist_pricing['volume']
#     price_dict = {'high': H,'low': L,'open': O,'close' : C, 'volume': V}
#     df = pd.DataFrame(price_dict, index=dates)    
#     return df

# def get_rt_pricing(symbol, freq='1d', prange='10d', cols=None):
#     data_dict = get_pricing(symbol, freq, prange)
#     df = build_px_struct(data_dict, freq)
#     cols = df.columns if cols is None else cols
#     return df[cols].dropna()

# def get_symbol_pricing(symbol, freq, cols=None):
#     path = config['pricing_path'].format(freq)
#     data_dict = json_load(path + json_ext.format(symbol))
#     df = build_px_struct(data_dict, freq)
#     cols = df.columns if cols is None else cols
#     return df[cols].dropna()

# def get_mults_pricing(symbols, freq, col='close'):
#     group_pricing = pd.DataFrame()
#     for n, t in enumerate(symbols):
#         print("Getting pricing for {0}".format(t))
#         sec_hp = get_symbol_pricing(t, freq,[col])
#         sec_hp.rename(columns={col: t}, inplace=True)
#         if n == 0: group_pricing = pd.DataFrame(sec_hp)
#         else: group_pricing[t] = sec_hp
#     return group_pricing

# def apply_std_boundaries(df, col='close', window=30, stds=2):
#     sma = df[col].rolling(window).mean()
#     smastd = df[col].rolling(window).std()
#     smaub = sma + smastd*stds
#     smalb = sma - smastd*stds
#     df['sma' + str(window)] = sma
#     df['sma' + str(window) + 'ub'] = smaub
#     df['sma' + str(window) + 'lb'] = smalb
#     df['sma' + str(window) + 'bw'] = smastd*stds / sma
#     return df

# def get_ind_index(closepx, freq='1d', tail=60, name='^IND'):
#     closepx = closepx.tail(tail)
#     memb_count = len(symbols)
#     eq_wgt = 1 / memb_count
#     closepx.set_index(pd.DatetimeIndex(closepx.index), inplace=True)
#     comp_indexed = (closepx.pct_change() + 1).cumprod()
#     comp_indexed.iloc[0] = 1
#     # comp_indexed.pct_change(), closepx.pct_change() # CHECK, should be the same
#     comp_indexed[name] = (comp_indexed * eq_wgt).sum(axis=1)
#     return comp_indexed

# def to_index_form(df, name):
#     dt_index = pd.DatetimeIndex(df.index)
#     idx = pd.DataFrame((df.pct_change() + 1).cumprod().values, 
#         index=dt_index, columns=[name])
#     idx.iloc[0] = 1
#     return idx

# def beta(df):
#     # first column is the market
#     X = df.values[:, [0]]
#     # prepend a column of ones for the intercept
#     X = np.concatenate([np.ones_like(X), X], axis=1)
#     # matrix algebra
#     b = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(df.values[:, 1:])
#     return pd.Series(b[1], df.columns[1:], name='Beta')

# def get_statspc_dates(df, treshold):
#     mask = df.abs() / df.std() > treshold
#     return df[mask][(df[mask].sum(axis=1) != 0).values]

# def eq_wgt_attribution(comp_idx, index_col, resample_period):
#     rp_delta = comp_idx.sub(comp_idx[index_col], axis='rows').iloc[:,:-1]
#     resampled = (rp_delta/len(rp_delta.columns)).resample(resample_period, level=0).sum()
#     return resampled[resampled.iloc[-1].sort_values(ascending=False).index]

# def rank_roll_perf(df, roll_window):
#     ind_ranked = df.round(2).rank(axis=1)
#     rolled_ranked = ind_ranked.rolling(roll_window).mean()
#     show = rolled_ranked.iloc[-1].sort_values().index.tolist()
#     return rolled_ranked[show]

# def get_left_right(alist, sl):
#     # mods large list into left / right chunks
#     be = len(alist) // sl
#     left, right = be + 1 if be * sl < len(alist) else be, be
#     return left, right



In [ ]:
# contextual information
symbol = 'VCYT'
market_etf = 'SPY'
freq, tail = '1d', 20 * 6
window, stds = 30, 2

dates = read_dates('quote') 
tgt_date = [dates[-1]] # last date saved in S3
# latest_quotes = load_csvs('quote_consol', tgt_date)
quotes = load_csvs('quote_consol', tgt_date)
profile = load_csvs('summary_detail', ['assetProfile'])
show = ['symbol','sector', 'industry']
industries = profile[show].sort_values(by='industry').industry.dropna().unique().tolist()
left, right = get_left_right(industries, sl)
sect_idx_ticker, ind_idx_ticker = '^SECT', '^IND'

### Retrieve pricing

In [ ]:
# TAKES LONG: all pricing loaded, should do only once
symbols_list = config['benchmarks'] + config['sectors'] + config['industries'] + config['companies'] + config['risk']
px_close = load_px_close('../tmp/', 'universe-px-ds', True).drop_duplicates().dropna(subset=['^GSPC'])
px_set = px_close[excl(symbols_list, ['LYFT'])]
px_close.info()

### Historical VIX

In [ ]:
# Apply BB to VIX
symbol = '^VIX'
# sec_pricing = get_symbol_pricing(symbol, freq, ['open', 'low', 'high', 'close'])
secpx = get_symbol_pricing(symbol, freq)
show = ['close']
df = apply_std_boundaries(secpx, show, 30, )
smacols = [x for x in df.columns if 'sma' in x]
show = ['close'] + smacols

# historical frequency
freq_col = 'close' # show[-1]
left, right = get_left_right(df, 2)
df[show[:-1]].iloc[-252:].plot(figsize=fs, ylim=[0, 50], grid=True);
df[show[:-1]].iloc[-right:].plot(figsize=fs, ylim=[0, 50], grid=True);
df[show[:-1]].iloc[:left].plot(figsize=fs, ylim=[0, 50], grid=True);

In [ ]:
# Frequency distribution of intervals
fd = freq_dist(df, freq_col, 252*10)
labels=[(round(x.left, 1), round(x.right,1)) for x in fd.index.values]

vix_freq = pd.concat([fd.to_frame(), fd.cumsum()], axis=1)
vix_freq.columns = ['hist_freq', 'cum_freq']
vix_freq.plot.bar(secondary_y='cum_freq')

In [ ]:
# Historical analysis when portfolio insurance is cheap
# When does vol crosses [low] bound at [x] std
show = ['close','sma30ub']
dates = df[df.close < df.sma30lb].index.tolist()
abovesma = pd.DataFrame([{'year':x.year, 'month':x.month, 'day':x.day} for x in dates])
abovesma.pivot_table(index='year', columns='month', aggfunc='count').replace(np.nan, '')

In [ ]:
# Last few dates the VIX is below trend
abovesma.tail(5)

### Sector overview

In [ ]:
# Pull sector ETFs for relative performance
symbols = config["sectors"]
closepx = px_set[symbols]
# openpx = get_mults_pricing(symbols, freq, 'open')
# ((closepx - openpx) / openpx * 100).std().sort_values().plot.barh()

comp_idx = get_ind_index(px_set[symbols], freq, tail, ind_idx_ticker)
SPY = get_symbol_pricing('SPY', freq, 'close').tail(tail)
comp_idx.drop(ind_idx_ticker, axis=1, inplace=True)
comp_idx['SPY'] = to_index_form(SPY, 'SPY')

In [ ]:
# Sector relative performance
cumulative = comp_idx.iloc[-1]
last_delta = cumulative - cumulative['SPY']
ctitle = f'{tail} day Sector ETF relative performance'
last_delta.sort_values(ascending=True).plot(
    kind='bar', label='index', title=ctitle);

In [ ]:
# Sector performance attribution
resampled = eq_wgt_attribution(comp_idx, 'SPY', 'W')
ctitle = 'Sector performance attribution'
ax = resampled.plot(
    kind='bar', stacked=True, figsize=(12,5), colormap=cmap, title=ctitle)
ax.set_xticklabels(resampled.index.strftime('%m-%d-%y'));
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# Sector performance rankings
roll_window = 4
rank_rolled = rank_roll_perf(resampled, roll_window)
left, right = get_left_right(rank_rolled.columns, sl)

ctitle = 'Underperforming sectors'
rank_rolled.iloc[:,:left].plot(figsize=fs, colormap=cmap, linewidth=lw, title=ctitle)\
    .legend(loc=loc, bbox_to_anchor=margin)

ctitle = 'Outperforming sectors'
rank_rolled.iloc[:,-right:].plot(figsize=fs, colormap=cmap, linewidth=lw, title=ctitle)\
    .legend(loc=loc, bbox_to_anchor=margin);

### Industry overview

In [ ]:
# Calculate industry indices
ind_indices = pd.DataFrame()
for n, ind in enumerate(industries):
    ind_id = shorten_name(ind)
    symbols = profile[profile.industry == ind].symbol.tolist()
    comp_idx = get_ind_index(px_set[symbols], '1d', tail, ind_id)
    ind_idx = comp_idx.iloc[:, -1:]
    if n == 0: ind_indices = pd.DataFrame(ind_idx)
    else: ind_indices[ind_id] = ind_idx

In [ ]:
q = int(ind_indices.shape[1]*.1)
focus_ind = list(ind_indices.iloc[-1, :].sort_values().iloc[:q].index) + list(ind_indices.iloc[-1, :].sort_values().iloc[-q:].index)

In [ ]:
# Industry performance relative ranking
SPY = get_symbol_pricing('SPY', freq, 'close').tail(tail)
ind_indices['SPY'] = to_index_form(SPY, 'SPY')
cumulative = ind_indices.iloc[-1]
last_delta = cumulative - cumulative['SPY']
# last_delta = cumulative
ctitle = 'Industry relative performance ranking'
last_delta[focus_ind].sort_values(ascending=False).plot(
    kind='bar', label='index', figsize=(12,5), title=ctitle);

In [ ]:
# Industry performance attribution
resampled = eq_wgt_attribution(ind_indices, 'SPY', 'W')
ctitle = 'Industry performance attribution'
ax = resampled[focus_ind].plot(kind='bar', stacked=True, 
                    figsize=(12,5), colormap=cmap, title=ctitle)
ax.set_xticklabels(resampled.index.strftime('%m-%d-%y'));
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# Industry performance rankings
resampled = eq_wgt_attribution(ind_indices, 'SPY', 'W')
rank_rolled = rank_roll_perf(resampled, 4)
left, right = get_left_right(rank_rolled.columns, sl)

ctitle = 'Underperforming sectors'
rank_rolled.iloc[:,:left].plot(
    figsize=fs, linewidth=lw, colormap=cmap, title=ctitle)\
        .legend(loc=loc, bbox_to_anchor=margin);
ctitle = 'Outperforming sectors'
rank_rolled.iloc[:,-right:].plot(
    figsize=fs, linewidth=lw, colormap=cmap, title=ctitle)\
        .legend(loc=loc, bbox_to_anchor=margin);

### Industry deep dive

In [ ]:
tgt_industry

In [ ]:
tgt_industry = industries[1]
symbols = profile[profile.industry == tgt_industry].symbol.tolist()
ind_id = shorten_name(tgt_industry)
[(shorten_name(ind[1]), ind) for ind in enumerate(industries)]

In [ ]:
# Industry overview charts
comp_idx = get_ind_index(px_set[symbols], '1d', tail, ind_id)
fig, axs = plt.subplots(ncols=1, nrows=3, constrained_layout=False, figsize=(12,15))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

print('{} analysis for {} days'.format(tgt_industry, tail))
(comp_idx[ind_id] - 1).plot(
    ax=axs[0], title='Industry performance');

comp_idx.pct_change().corr()[ind_id].iloc[:-1].sort_values(ascending=False).plot(
    kind='bar', ax=axs[1], title='Component correlation');

(comp_idx.iloc[-1].sort_values(ascending=False) - 1).plot(
    kind='bar', ax=axs[2], title='Relative performance');

plt.show()

In [ ]:
# Industry components performance attribution
resampled = eq_wgt_attribution(comp_idx, ind_id, 'W')
left, right = get_left_right(resampled.columns, 2)
show = resampled.columns[:(left)].tolist() + resampled.columns[-right:].tolist()
ctitle = 'Equal weight attribution for top/bottom {} components'.format(left, ind_id)
ax = resampled[show].plot(
    colormap=cmap, kind='bar', stacked=True, figsize=(12,5), title=ctitle)
ax.set_xticklabels(resampled.index.strftime('%m-%d-%y'));
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# Industry components performance ranking
rank_rolled = rank_roll_perf(resampled, 4)
ctitle = 'Underperforming names'
rank_rolled.iloc[:,:left].plot(
    figsize=fs, linewidth=lw, colormap=cmap, title=ctitle)\
        .legend(loc=loc, bbox_to_anchor=margin);
ctitle = 'Outperforming names'
rank_rolled.iloc[:,-right:].plot(
    figsize=fs, linewidth=lw, colormap=cmap, title=ctitle)\
        .legend(loc=loc, bbox_to_anchor=margin);

### Company relative to industry, sector and market performance
What's a statistically significant move?<br>
De-correlate vs. industry, sector, and market<br>
List key dates<br>
Create function to highlight relevant idiosincratic behaviour<br>

In [ ]:
symbol = 'VRSK'
row = profile[profile.symbol == symbol]
sec_sector, sec_industry = row.iloc[0].sector, row.iloc[0].industry
sec_sector, sec_industry

In [ ]:
sec_index = to_index_form(
    get_symbol_pricing(symbol, freq, 'close').tail(tail),
    symbol)

symbols = profile[profile.industry == sec_industry].symbol.tolist()
industry_index = get_ind_index(
    px_set[symbols], '1d', tail, ind_idx_ticker)[[ind_idx_ticker]]

symbols = profile[profile.sector == sec_sector].symbol.tolist()
sector_index = get_ind_index(
    px_set[symbols], '1d', tail, sect_idx_ticker)[[sect_idx_ticker]]

market_index = to_index_form(
    get_symbol_pricing(market_etf, freq, 'close').tail(tail),
    market_etf)

index_df = pd.DataFrame()
index_df = index_df.append(sec_index)
index_df[ind_idx_ticker] = industry_index
index_df[sect_idx_ticker] = sector_index
index_df[market_etf] = market_index

In [ ]:
index_df_pc = index_df.pct_change().iloc[1:]
print('{} days SPY beta'.format(len(index_df_pc)))
beta(index_df_pc[[market_etf,symbol,ind_idx_ticker,sect_idx_ticker]])

In [ ]:
roll_window = 30
roll_vol_df = roll_vol(index_df_pc, roll_window)
print(f'{roll_window} days rolling volatility')
roll_vol_df.plot();
roll_vol_df.tail()

In [ ]:
print(f'{len(index_df_pc)} days correlation matrix')
index_df_pc.corr()

In [ ]:
# show statistical significant moves for further key factors study
treshold = 2
statspc_df = get_statspc_dates(index_df_pc, treshold)
show = statspc_df.columns[:2]
filtered = statspc_df[statspc_df[show].sum(axis=1).abs() > 0][show].sort_index(ascending=False)
ax = filtered.plot(kind='barh')
ax.set_yticklabels(filtered.index.strftime('%Y-%m-%d'))
index_df.plot()
for d in filtered.index: plt.axvline(x=d, c='k', linewidth=0.5)

plt.show()
filtered

### Single security pricing, all fields

In [ ]:
# Retrieves historical pricing
secpx = get_symbol_pricing(symbol, freq)
secpx.tail(5)

In [ ]:
df = apply_std_boundaries(secpx, 'close', window, stds)
smacols = [x for x in df.columns if 'sma' in x]
show = ['close'] + smacols

df[show[:-1]].tail(tail).plot(secondary_y=freq_col, figsize=(12,8))
freq_col = 'sma' + str(window) + 'bw'
fd = freq_dist(df, freq_col, tail)
pd.DataFrame(fd).plot.barh(label='index');

### Long exposure correlations

In [ ]:
long_symbols = ['1810.HK', '9984.T', 'BABA', 'FB', 'GOOG', 'XOP']
nbr_pos = len(long_symbols)
# long_weights = [1/nbr_pos for x in long_symbols]
long_weights = [10.3, 3.7, 60.4, 22.3, 8.4, 3.18]
long_wv = np.array(long_weights) / 100

In [ ]:
long_px = px_set[
    long_symbols + \
    config['benchmarks'] + \
    config['sectors']
    ].tail(tail)

# corr_mtrx = long_px.pct_change().corr()
corr_mtrx = long_px.pct_change().corr()
cov_mtrx = long_px.pct_change().cov() * 252

port_corr_mtrx = corr_mtrx.iloc[:nbr_pos, :nbr_pos]
port_cov_mtrx = cov_mtrx.iloc[:nbr_pos, :nbr_pos]
# tri_corr_mtrx = (port_corr_mtrx * np.tri(nbr_pos,nbr_pos, -1))
# cross_corr = np.dot(long_wv.T, np.dot(tri_corr_mtrx, long_wv))

cross_corr = np.dot(long_wv.T, np.dot(port_corr_mtrx, long_wv))
cross_vol = np.sqrt(np.dot(long_wv.T, np.dot(port_cov_mtrx, long_wv)))
ctitle = 'Portfolio {0} correlation {1} annualized volatility'.format(
    round(cross_corr,2), round(cross_vol,2))
print(ctitle)
port_corr_mtrx

In [ ]:
top_corr_items = 3
corr_mtrx_bench = corr_mtrx.loc[long_symbols].T.iloc[nbr_pos:]
best_pairs = [(x, corr_mtrx_bench[corr_mtrx_bench[x] < 1][x].sort_values(ascending=False).index.tolist()[:top_corr_items]) for x in long_symbols]
best_pairs, corr_mtrx_bench

In [ ]:
cols = []; [cols.extend(y) for x, y in best_pairs]
cols = list(set(cols))
port_exp = pd.DataFrame([], index=long_symbols, columns=cols)
# port_exp.loc['FB', ['XLU','FXI']] = [1, 2]
for x, y in best_pairs: port_exp.loc[x, y] = corr_mtrx_bench.T.loc[x, y]
port_exp.replace(np.nan, '')

In [ ]:
# portfolio greatest exposure
pd.Series(
    np.dot(port_exp.replace(np.nan, 0).T, long_wv.T), index=cols)\
    .sort_values(ascending=False)

In [ ]:
# Rolling correlation
roll_window = 20
missing_px = (long_px[long_symbols].count() < len(long_px[long_symbols])).tolist()
items = [best_pairs[x] for x in range(len(missing_px)) 
         if not missing_px[x]]
for x, y in items:
    title = '{} {} day rolling correlations'.format(x, roll_window)
    long_px[x]\
        .rolling(roll_window).corr(long_px[y])\
        .plot(title=title, rot=45)
# long_px.index.astype(date)

### Benchmark option pricing

In [ ]:
underlyingSymbol = 'QQQ'

In [ ]:
!scp -i ~vveiga/.ssh/qc_infra.pem ubuntu@54.191.249.196:/home/ubuntu/inception/tmp/universe-px-ds ../tmp/universe-px-ds
px_close = load_px_close('../tmp/', 'universe-px-ds', True).drop_duplicates().dropna(subset=['^GSPC'])
px_close.info()

In [ ]:
options = load_csvs('option_consol', ['2019-04-16'])

In [ ]:
quotes.set_index('symbol', drop=False, inplace=True)
options.set_index('contractSymbol', drop=False, inplace=True)

In [ ]:
mask = options['underlyingSymbol'] == underlyingSymbol
atm_cont = np.abs((options.loc[mask, 'strike'] - quotes.loc['QQQ', 'regularMarketPrice'])).sort_values().index[0]
atm_strike = options.loc[atm_cont, 'strike']
cut_off = options.loc[mask].openInterest.sort_values().quantile(0.25)
mask = mask \
    & (options['strike'] == atm_strike) \
    & (options['type'] == 'call') \
    & (options['openInterest'] >= cut_off)
opt_subset = options.loc[mask]
opt_subset.loc[:, 'daysExpiration'] = -(np.datetime64(datetime.today()) - opt_subset.expiration).dt.days.values
opt_subset.loc[:, 'premiumDay'] = opt_subset.loc[:, 'lastPrice'].T / -(np.datetime64(datetime.today()) - opt_subset.expiration).dt.days.values
opt_subset.loc[:, ['expiration', 'impliedVolatility', 'lastPrice', 'daysExpiration', 'premiumDay']]

In [ ]:
opt_subset[['impliedVolatility', 'premiumDay']].plot.scatter(x='impliedVolatility', y='premiumDay')

In [ ]:
roll_vol(px_close[underlyingSymbol].pct_change(), 60).tail(252).plot()

In [ ]:
options = load_csvs('option_consol', tgt_date)
cols_to_date = ['expiration', 'lastTradeDate', 'storeDate']
for c in cols_to_date: options[c] = options[c].apply(lambda x: datetime.fromtimestamp(x))
options.info()
# options.iloc[3]

In [ ]:
# find expirations with more than 90 days
def option_IV_atm(options, symbol, calDaysOut, minVolume, optnType, nbrStrikes):
    symbols = config['benchmarks'] + config['sectors']
    next_dt = np.datetime64(datetime.today() + pd.Timedelta(calDaysOut, unit='d'))
    uniq_expirations = options.expiration.sort_values().unique()
    tgt_expirations = uniq_expirations[uniq_expirations > next_dt]
    mask = (options.underlyingSymbol.isin(symbols)) &\
        (options.volume > minVolume) &\
        options.expiration.isin(tgt_expirations) &\
        (options.type == optnType)
    subset = options[mask]
    last_price = px_set[symbols].iloc[-1][symbol]
    contracts = subset[subset.underlyingSymbol == symbol]
#     atm_contract = (contracts.strike / last_price - 1).abs().idxmin()
    atm_contract = (contracts.strike / last_price - 1).abs().\
        sort_values().index[:nbrStrikes]
    return contracts.loc[atm_contract].sort_values(by='strike')

In [ ]:
# shows implied volatility for all benchmarks
symbols = config['benchmarks'] + config['sectors']
b_IVs = pd.DataFrame()
for s in symbols:
#     print('Current price for {}: {}'.format(s, px_set[s].iloc[-1]))
    b_IVs = b_IVs.append(option_IV_atm(options, s, 90, 50, 'put', 1))
show = ['underlyingSymbol', 'expiration', 
        'strike', 'impliedVolatility', 
        'openInterest', 'volume','lastTradeDate']
# b_IVs[show].plot(kind='bar', x=show[0], y=show[1])
b_IVs[show].sort_values(by='expiration')

In [ ]:
# issue here is that rows could have diff. expirations
# shows mutiple implied volatility for one benchmark
symbol = 'XLK'
contract_list = option_IV_atm(options, symbol, 90, 50, 'put', 5)
contract_list[show].sort_values(by='expiration')

In [ ]:
# PENDING: shows historical implied volatility for one benchmark
symbol = 'SPY'
b_IVs = pd.DataFrame()
for d in [dates[-20:]]:
    options = load_csvs('option_consol', d)
    cols_to_date = ['expiration', 'lastTradeDate', 'storeDate']
    for c in cols_to_date: 
        options[c] = options[c].apply(lambda x: datetime.fromtimestamp(x))
    option_IV_atm(options, symbol, 60, 50, 'put', 1)
b_IVs[show]

### Other functions

In [ ]:
df['hlSpread'] = (df['high'] - df['low']) / df['open']
df['coSpread'] = (df['close'] - df['open']) / df['open']
df['1mFwdRet'] = df['close'].shift(-20) / df['close'] - 1
df.coSpread.hist()
# df[df.coSpread < -0.075]

In [ ]:
path = config['pricing_path'].format('1d')
data_dict = json_load(path + json_ext.format(symbol))
print(data_dict.keys())
data_dict['meta']
dates = [datetime.fromtimestamp(x) for x in data_dict['timestamp']]
indicators_quote_0 = data_dict['indicators']['quote'][0]
H = indicators_quote_0['high']
L = indicators_quote_0['low']
O = indicators_quote_0['open']
C = indicators_quote_0['close']
V = indicators_quote_0['volume']

In [ ]:
price_dict = {'high': H,'low': L,'open': O,'close' : C, 'volume': V}
df = pd.DataFrame(price_dict, index=dates)

In [ ]:
fields = ['open', 'close', 'high', 'low']
# fields = ['volume']
tgt_df = df[fields]
tgt_df.groupby(tgt_df.index.hour).mean().plot()
# tgt_df.plot()